<a href="https://colab.research.google.com/github/eyaler/avatars4all/blob/master/yarok.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# U^2-Net Video Green Screen
## https://github.com/NathanUA/U-2-Net
### Made just a little bit more accessible by Eyal Gruss (https://eyalgruss.com, eyalgruss@gmail.com)
##### List of more generative tools: https://j.mp/generativetools

In [ ]:
#@title Setup
%cd /content
!git clone --depth 1 https://github.com/NathanUA/U-2-Net
!mkdir -p /content/U-2-Net/saved_models/u2net
%cd /content/U-2-Net/saved_models/u2net
!wget --no-check-certificate -nc https://eyalgruss.com/fomm/u2net.pth
%cd /content
!pip install -U youtube-dl
!pip install -U imageio
!pip install -U imageio-ffmpeg
!pip install -U imageio-ffmpeg

In [ ]:
#@title Get the foreground video and background image from the web
#@markdown 1. You can change the URLs to your **own** stuff!
#@markdown 2. For the background image you can use the drop-down menu to choose a **solid color**
#@markdown 3. Alternatively, you can upload **local** files in the next cells

video_url = 'https://www.youtube.com/watch?v=kMpnwIGDQvU' #@param {type:"string"}
image_url = 'https://cdnb.artstation.com/p/assets/images/images/012/476/185/large/rafael-de-jongh-synthwave-neon-80s-background-marmoset-revamp-compressed.jpg' #@param ['https://cdnb.artstation.com/p/assets/images/images/012/476/185/large/rafael-de-jongh-synthwave-neon-80s-background-marmoset-revamp-compressed.jpg', 'Black', 'White', 'Green'] {allow-input: true}

if video_url:
  !rm -f /content/video.mp4
  !youtube-dl -f 'bestvideo[ext=mp4][height<=360]+bestaudio[ext=m4a]/mp4[height<=360]/mp4' '$video_url' --merge-output-format mp4 -o /content/video
  duration_seconds = None
  params = '' 

if '://' in image_url:
  !wget '$image_url' -O /content/image

In [ ]:
#@title Optionally upload foreground video { run: "auto" }
manually_upload_video = False #@param {type:"boolean"}
if manually_upload_video:
  from google.colab import files
  import shutil

  %cd /content/sample_data
  try:
    uploaded = files.upload()
  except Exception as e:
    %cd /content
    raise e

  for fn in uploaded:
    shutil.move('/content/sample_data/'+fn, '/content/video')
    break
  params = ''

In [ ]:
#@title Optionally upload local background image { run: "auto" }
manually_upload_image = False #@param {type:"boolean"}
if manually_upload_image:
  from google.colab import files
  import shutil

  %cd /content/sample_data
  try:
    uploaded = files.upload()
  except Exception as e:
    %cd /content
    raise e

  for fn in uploaded:
    shutil.move('/content/sample_data/'+fn, '/content/image')
    break
    image_url = None

In [ ]:
#@title Optionally shorten video
start_seconds = 0 #@param {type:"number"}
duration_seconds =  60#@param {type:"number"}
start_seconds = max(start_seconds,0)
duration_seconds = max(duration_seconds,0)
params = '-ss %i -t %i'%(start_seconds, duration_seconds)

In [ ]:
#@title Green Screen It!
#@markdown If you ran out of RAM this means that the video is too large. You can shorten it above.

frames_dir = '/content/U-2-Net/test_data/test_images'
mask_dir = '/content/U-2-Net/test_data/u2net_results'
!rm -rf $frames_dir
!mkdir -p $frames_dir
!rm -rf $mask_dir
!mkdir -p $mask_dir
!ffmpeg $params -i /content/video.mp4 $frames_dir/frame_%05d.png

from skimage.transform import resize
import imageio
import numpy as np
from skimage import img_as_ubyte
import os
import warnings
warnings.filterwarnings("ignore")

def fix_dims(im):
    if im.ndim == 2:
        im = np.tile(im[..., None], [1, 1, 3])
    return im[...,:3]

target_image = imageio.imread(frames_dir+'/frame_00001.png')
if image_url == 'Black':
  source_image = np.full_like(target_image, 0)
elif image_url == 'White':
  source_image = np.full_like(target_image, 1)
elif image_url == 'Green':
  source_image = np.full_like(target_image, [0,1,0])
else:
  source_image = imageio.imread('/content/image')
  source_image = fix_dims(source_image)
  source_image = resize(source_image, target_image.shape[:2])

%cd /content/U-2-Net
!python /content/U-2-Net/u2net_test.py

combined = []
for file in sorted(os.listdir(frames_dir)):
  try:
    im = imageio.imread(frames_dir+'/'+file)/255
    mask = imageio.imread(mask_dir+'/'+file)/255
  except Exception:
    continue
  combined.append(source_image*(1-mask)+im*mask)

with imageio.get_reader('/content/video.mp4') as reader:
  fps = reader.get_meta_data()['fps']

imageio.mimsave('/content/combined.mp4', [img_as_ubyte(frame) for frame in combined], fps=fps, macro_block_size=None)
!ffmpeg $params -i /content/combined.mp4 $params -i /content/video.mp4 -c copy -map 0:v -map 1:a /content/final.mp4 -y
#video can be downloaded from /content/final.mp4

from IPython.display import HTML, clear_output
from base64 import b64encode

clear_output()
with open('/content/final.mp4', 'rb') as f:
  data_url = "data:video/mp4;base64," + b64encode(f.read()).decode()
display(HTML("""
<video width=600 controls autoplay loop>
      <source src="%s" type="video/mp4">
</video>""" % data_url))

In [ ]:
#@title Download
#@markdown 1. If it fails try running this cell again.
#@markdown 2. Alternatively, you can manually download "final.mp4" from the folder on the left (click "Refresh" if missing).

print() #see https://github.com/googlecolab/colabtools/issues/468
from google.colab import files
files.download('/content/final.mp4') #fails for Firefox private window